In [27]:
import pandas as pd
import psycopg2
import matplotlib
import numpy as np
#from connect import connect_DB #import custom connection script

#connector = connect_DB() #instantiate connector class
conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
cur = conn.cursor()
#list of HMDA years to use
#property type appears in 2004
#FIXME reload and clean 2004
#FIXME: add property type into 2004 and beyond?
hmda_tables = ['hmdalar2000', 'hmdalar2001', 'hmdalar2002', 'hmdalar2003', 'hmdalar2004', 'hmdalar2005', 'hmdalar2006',
'hmdalar2007', 'hmdalar2008', 'hmdalar2009', 'hmdalar2010', 'hmdalar2011', 'hmdalar2012', 'hmdalar2013', 'hmdalar2014']


#get county level data
#count of loans
#average and median dollar value of loans
#dollar volume of loan amount


In [29]:
#selects application information for single family homes
first = True
for table in hmda_tables:

    SQL = """
        SELECT year, state, county,
        round(avg(amount::integer),2) as loan_average,
        round(avg(income::integer),2) as income_average,
        count(concat(agency, rid)) as count_of_apps,
        sum(amount::integer) as app_volume
        FROM {table}
        WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')

        AND income not like '%NA%'
        AND income not like '%na% '
        AND amount not like '%NA%'
        AND amount not like '%na%'

        GROUP BY state, county, year """

    SQL = SQL.format(table=table, type='1')
    #cur.execute(SQL)

    #data = cur.fetchall() #retrieve data from query
    df = pd.read_sql_query(SQL, conn)
    if first:
        out_file = df
    else:
        out_file = out_file.concat(df)
    print df.head()


out_file.to_csv('county_aggregates.csv')



   year state county  loan_average  income_average  count_of_apps  app_volume
0  2000    13    005         42.91           33.63            334       14332
1  2000    25    025        172.26           92.79          23620     4068697
2  2000    55    065         61.86           45.41            422       26106
3  2000    38    023         28.00           43.50              6         168
4  2000    51    037         59.77           40.45            314       18767
   year state county  loan_average  income_average  count_of_apps  app_volume
0  2001    38    049         47.77           49.31            127        6067
1  2001    47    151         53.79           35.79            964       51858
2  2001    31    111         68.30           57.33           1321       90224
3  2001    21    227         92.14           65.20           5679      523286
4  2001    37    109        102.00           58.97           5446      555513
   year state county  loan_average  income_average  count_of_app

In [31]:
county_agg_df = out_file.copy()

In [ ]:
#selects originated loan information for single family homes
first = True
for table in hmda_tables[4:]:
    SQL = """
WITH rates as (
SELECT year, state, county,
    count(rate_spread) as has_rs,
    avg(rate_spread::numeric) as avg_rate_spread
    
    FROM {table}
    
    WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND property_type = '{type}'
        AND action = '1'
        AND lien = '1'
        AND rate_spread != 'NA   '
    GROUP BY year, state, county),

rates2 as (
    SELECT year, state, county,
    count(rate_spread) as no_rs
    
    FROM {table}
    
    WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND property_type = '{type}'
        AND action = '1'
        AND lien = '1'
        AND rate_spread = 'NA   '
GROUP BY year, state, county),

county_agg_orig AS (
    SELECT year, state, county,
        round(avg(amount::integer),2) as loan_average,
        round(avg(income::integer),2) as income_average,
        count(concat(agency, rid)) as count_orig,
        sum(amount::integer) as orig_value

    FROM {table}
    
    WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND property_type = '{type}'
        AND action = '1'
        AND lien = '1'
        AND income not like '%NA%'
        AND income not like '%na% '
        AND amount not like '%NA%'
        AND amount not like '%na%'
        GROUP BY state, county, year ),

county_agg_apps AS (SELECT year, state, county,
        round(avg(amount::integer),2) as loan_average,
        round(avg(income::integer),2) as income_average,
        count(concat(agency, rid)) as count_apps,
        sum(amount::integer) as app_value
        FROM {table}
        WHERE loan_type = '1'
        AND loan_purpose in ('1', '3')
        AND property_type = '{type}'
        AND action != '1'
        AND lien = '1'
        AND income not like '%NA%'
        AND income not like '%na% '
        AND amount not like '%NA%'
        AND amount not like '%na%'
        GROUP BY state, county, year )
        
SELECT rates.year, rates.state, rates.county, rates.has_rs, rates2.no_rs, 
round(rates.avg_rate_spread, 2) as avg_rs, 
round(((rates.has_rs::numeric / (rates2.no_rs::numeric + rates.has_rs::numeric))*100),2) AS rs_ratio,
county_agg_orig.loan_average, 
county_agg_orig.income_average, 
county_agg_orig.count_orig, 
county_agg_orig.orig_value,
round(((county_agg_orig.loan_average::numeric/.8) / county_agg_orig.income_average::numeric),2) AS income_multiple,
county_agg_apps.loan_average AS app_loan_avg,
county_agg_apps.income_average AS app_income_avg,
county_agg_apps.count_apps AS count_apps,
county_agg_apps.app_value,
ROUND(((county_agg_orig.count_orig::numeric / (county_agg_orig.count_orig::numeric + county_agg_apps.count_apps::numeric))*100),2) AS orig_rate


FROM rates
JOIN rates2
ON concat(rates.state, rates.county) = concat(rates2.state, rates2.county)
JOIN county_agg_orig
ON concat(rates.state, rates.county) = concat(county_agg_orig.state, county_agg_orig.county)
JOIN county_agg_apps
ON concat(rates.state, rates.county) = concat(county_agg_apps.state, county_agg_apps.county);
"""

    SQL = SQL.format(table=table, type='1')
    df = pd.read_sql_query(SQL, conn)
    if first:
        out_file = df
    else:
        out_file = out_file.concat(df)
    print df.head()
out_file.to_csv('county_aggregates_post_2004.csv')
#export as single csv for all years
#create table to hold data
#insert data

In [ ]:
out_file.tail()